In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np

In [2]:
EXPERIMENT = True

In [3]:
full_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/'

In [4]:
base_dir = full_data_dir
folder = 'v1.0-trainval'

In [5]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [6]:
files = os.listdir(os.path.join(full_data_dir, 'experiment_data'))

In [7]:
import math
from typing import List

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw)

def get_heading_from_north(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw - (math.pi / 2))

rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
def get_camera_heading(rotation):
    return -get_heading(rot.rotate(rotation)) + math.pi / 2
    
def get_camera_position(
    camera_translation: List[float],
    ego_translation: List[float],
    ego_rotation: List[float]
) -> np.ndarray:
    rotated_offset = Quaternion(ego_rotation) \
        .inverse \
        .rotate(np.array(camera_translation))
    return np.array(ego_translation) + rotated_offset

In [8]:
files_set = set(files)

In [9]:
if EXPERIMENT:
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
# sample_data_filter = [s for s in sample_data_json if 'CAM_FRONT/' in s['filename']]

In [10]:
sample_data_filter[0]

{'token': '3a6f2bbccb76463f871c5720e7aae454',
 'sample_token': 'da3c26d4d7c44efc83d854be98efb5d7',
 'ego_pose_token': '3a6f2bbccb76463f871c5720e7aae454',
 'calibrated_sensor_token': 'bff1f6294cbc4b61a07f2d85b9b5f391',
 'timestamp': 1533153257412404,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n008-2018-08-01-15-52-19-0400__CAM_FRONT__1533153257412404.jpg',
 'prev': '6e250417c6c04b51b17266c9b3b12e63',
 'next': '12985a804ecb49faacb355dff0dd98eb'}

In [11]:
sample_tokens = set([s['sample_token'] for s in sample_data_filter])

In [12]:
ego_pose_tokens = set([s['ego_pose_token'] for s in sample_data_filter])

In [13]:
calibrated_sensor_tokens = set([s['calibrated_sensor_token'] for s in sample_data_filter])

In [14]:
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

707

In [15]:
scene_tokens = set([s['scene_token'] for s in sample_filter])

In [16]:
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

169

In [17]:
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

707

In [18]:
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

169

In [19]:
sample_map = {
    s['sample_token']: s
    for s in sample_filter
}
calibrated_sensor_map = {
    c['calibrated_sensor_token']: c
    for c in calibrated_sensor_filter
}
ego_pose_map = {
    e['ego_pose_token']: e
    for e in ego_pose_filter
}
scene_map = {
    s['scene_token']: s
    for s in scene_filter
}

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene_map[sample['scene_token']],
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalizeAngle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation_absolute': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation']
        )
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

707

In [20]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

21226

In [21]:
instance_tokens = set([
    sa['instance_token']
    for sa in sample_annotation_filter
])

instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

6862

In [22]:
category_tokens = set([
    i['category_token']
    for i in instance_filter
])

category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

23

In [23]:
instance_map = {
    i['instance_token']: i
    for i in instance_filter
}
category_map = {
    c['category_token']: c
    for c in category_filter
}

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

21226

In [24]:
sample_data_final = pd.DataFrame(sample_data_res)

In [25]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)

In [26]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)

In [27]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [28]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [29]:
print(len(sample_data_final))

707


In [30]:
print(len(sample_annotation_final))

21226


In [31]:
if EXPERIMENT:
    suffix = '_experiment'
else:
    suffix = ''

In [32]:
with open(os.path.join(base_dir, f"sample_data{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(base_dir, f"annotation{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [33]:
sample_data_final.to_csv(os.path.join(base_dir, f"sample_data{suffix}.csv"), index=False)

In [34]:
sample_annotation_final.to_csv(os.path.join(base_dir, f"annotation{suffix}.csv"), index=False)

In [35]:
calibrated_sensor = [
    {
        **c,
        **[s for s in sensor_json if s['token'] == c['sensor_token']][0]
    }
    for c in calibrated_sensor_json
]
calibrated_sensor = [
    c
    for c in calibrated_sensor
    if c['modality'] == 'camera'
]
rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
[
    normalizeAngle(get_heading(rot.rotate(Quaternion(c['rotation'])))) * 180 / math.pi
    for c in calibrated_sensor
    if c['channel'] == "CAM_FRONT"
]

[89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,


In [36]:
sample_data_final

,token,sample_token,timestamp,is_key_frame,filename,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,ego_heading,camera_heading,camera_translation_absolute,frame_order
3,56c4ae3a0c5c45adad2e28e23c69bdfe,bdb6983a1de646f7b99519c7b4d496b1,1533153253912404,True,samples/CAM_FRONT/n008-2018-08-01-15-52-19-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[2192.9708906325777, 792.113215216536, 0.0]","[-0.05089727661959409, -0.0014913268606285026,...",scene-0062,95.835924,96.423821,"[2191.2518862890024, 792.2643420849427, 1.4907...",1
2,6ddf1ce90e69442fa8437b0aa7b20269,e691aa5350704584803ae3b3aebcd6e9,1533153254412404,True,samples/CAM_FRONT/n008-2018-08-01-15-52-19-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[2188.7205342395423, 791.6347178701818, 0.0]","[-0.07324861527338412, -0.0022800825929781736,...",scene-0062,98.402860,98.990758,"[2187.0080074743464, 791.861399861563, 1.48860...",2
0,3a6f2bbccb76463f871c5720e7aae454,da3c26d4d7c44efc83d854be98efb5d7,1533153257412404,True,samples/CAM_FRONT/n008-2018-08-01-15-52-19-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[2164.081820635511, 788.9733057859266, 0.0]","[-0.02006851799168256, 0.0019368004660924336, ...",scene-0062,92.296564,92.884462,"[2162.365983318183, 788.99612065799, 1.5018230...",3
1,6184b3510235467c92763c9955a6ce5d,d3d4393a9bfb4441b7fb9aedd3330c29,1533153258862404,True,samples/CAM_FRONT/n008-2018-08-01-15-52-19-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[2152.2261967146796, 788.5965807473574, 0.0]","[-0.015188090498247951, 0.0024902016656368634,...",scene-0062,91.735766,92.323664,"[2150.5115851321348, 788.5964566721132, 1.5033...",4
6,f45abc7d6edc488b84073059f2c1832a,381d9dfde64041cbac749fd66627e3aa,1533153275262404,True,samples/CAM_FRONT/n008-2018-08-01-15-52-19-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[2018.6485955520575, 785.4581022955552, 0.0]","[0.014340582029098702, -0.0013416486779454797,...",scene-0063,91.641321,92.229219,"[2016.9306424773058, 785.4652516992435, 1.4995...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,e0fcb740133a46c1b7e61c29d406b12f,72fb4e43b4a44e70b46e25d46b4c296f,1537298183262404,True,samples/CAM_FRONT/n008-2018-09-18-15-12-01-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[620.6429459866663, 683.6970308100175, 0.0]","[-0.35210577148901345, -0.0024953361069967096,...",scene-0913,48.768355,49.356253,"[619.3540346833465, 682.5718347945526, 1.50755...",6
702,b1bf92905d62408492277c755490ce7b,2e547a31b1564e6a8e8c8b44e987dd71,1537298183762404,True,samples/CAM_FRONT/n008-2018-09-18-15-12-01-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[617.2863501393006, 686.6932403363334, 0.0]","[-0.355043133108889, -0.0038496629011835343, -...",scene-0913,48.407868,48.995766,"[616.0097845306409, 685.5577237059574, 1.51032...",7
706,d7a6b49852e84d778af2f6af36223fb4,7b44e1d54b7e4a59bad57959aeaf3a70,1537298184162404,True,samples/CAM_FRONT/n008-2018-09-18-15-12-01-040...,"[1.72200568478, 0.00475453292289, 1.49491291905]","[0.5077241387638071, -0.4973392230703816, 0.49...","[[1252.8131021185304, 0.0, 826.588114781398], ...","[614.6398627077735, 689.1072350356118, 0.0]","[-0.3581905686362418, -0.002